In [16]:
import os
import pandas as pd

In [ ]:
data = []

# Walk through healthy images
for root, dirs, files in os.walk('All_healthy'):
    for file in files:
        if file.endswith(('.jpg', '.jpeg', '.png')):
            filepath = os.path.join(root, file)
            
            data.append((filepath, 0))  # Label 0 for healthy

# Walk through diseased images
for root, dirs, files in os.walk('diseased'):
    for file in files:
        if file.endswith(('.jpg', '.jpeg', '.png')):
            filepath = os.path.join(root, file)
            data.append((filepath, 1))  # Label 1 for diseased

# Create DataFrame
df = pd.DataFrame(data, columns=['filename', 'label'])
df = df.sample(frac=1).reset_index(drop=True)  # Shuffle the data
df.to_csv('plant_health_labels.csv', index=False)

In [22]:
import os

data = []

root_folders = ['All_healthy', 'diseased']

for root in root_folders:
    for subfolder in os.listdir(root):
        folder_path = os.path.join(root, subfolder)
        if not os.path.isdir(folder_path):
            continue
        label = f"{root}_{subfolder}"
        for file in os.listdir(folder_path):
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                filepath = os.path.join(folder_path, file)
                data.append((filepath, label))

# View first few
print(data[:5])


[('All_healthy\\Cashew_healthy\\healthy1000_.jpg', 'All_healthy_Cashew_healthy'), ('All_healthy\\Cashew_healthy\\healthy1001_.jpg', 'All_healthy_Cashew_healthy'), ('All_healthy\\Cashew_healthy\\healthy1002_.jpg', 'All_healthy_Cashew_healthy'), ('All_healthy\\Cashew_healthy\\healthy1003_.jpg', 'All_healthy_Cashew_healthy'), ('All_healthy\\Cashew_healthy\\healthy1004_.jpg', 'All_healthy_Cashew_healthy')]


In [23]:
import pandas as pd

data = pd.DataFrame(data, columns=["filename", "label"])
data

,filename,label
0,All_healthy\Cashew_healthy\healthy1000_.jpg,All_healthy_Cashew_healthy
1,All_healthy\Cashew_healthy\healthy1001_.jpg,All_healthy_Cashew_healthy
2,All_healthy\Cashew_healthy\healthy1002_.jpg,All_healthy_Cashew_healthy
3,All_healthy\Cashew_healthy\healthy1003_.jpg,All_healthy_Cashew_healthy
4,All_healthy\Cashew_healthy\healthy1004_.jpg,All_healthy_Cashew_healthy
...,...,...
25215,diseased\Tomato_verticulium wilt\verticulium w...,diseased_Tomato_verticulium wilt
25216,diseased\Tomato_verticulium wilt\verticulium w...,diseased_Tomato_verticulium wilt
25217,diseased\Tomato_verticulium wilt\verticulium w...,diseased_Tomato_verticulium wilt
25218,diseased\Tomato_verticulium wilt\verticulium w...,diseased_Tomato_verticulium wilt


In [19]:
!pip install tensorflow

In [49]:
from PIL import Image, UnidentifiedImageError
import os
import shutil
import pandas as pd

bad_image_dir = 'bad_images'
os.makedirs(bad_image_dir, exist_ok=True)

def is_fully_valid_image(path):
    try:
        with Image.open(path) as img:
            img = img.convert("RGB")  # Ensure 3-channel
            img.resize((224, 224))    # Test actual resizing
        return True
    except (UnidentifiedImageError, OSError, ValueError) as e:
        print(f"[BAD] {path} -- {e}")
        return False

valid_paths = []
for path in data['filename']:
    if is_fully_valid_image(path):
        valid_paths.append(path)
    else:
        try:
            shutil.move(path, os.path.join(bad_image_dir, os.path.basename(path)))
        except Exception as e:
            print(f"Failed to move {path}: {e}")

# Final clean DataFrame
data = data[data['filename'].isin(valid_paths)].reset_index(drop=True)




[BAD] All_healthy/Maize_healthy/healthy189_.jpg -- broken data stream when reading image file
[BAD] All_healthy/Maize_healthy/healthy87_.jpg -- broken data stream when reading image file
[BAD] All_healthy/Tomato_healthy/healthy442_.jpg -- broken data stream when reading image file
[BAD] All_healthy/Tomato_healthy/healthy76_.jpg -- broken data stream when reading image file
[BAD] diseased/Maize_leaf beetle/leaf beetle206_.jpg -- broken data stream when reading image file
[BAD] diseased/Maize_leaf beetle/leaf beetle457_.jpg -- broken data stream when reading image file
[BAD] diseased/Maize_leaf beetle/leaf beetle572_.jpg -- broken data stream when reading image file
[BAD] diseased/Maize_leaf beetle/leaf beetle68_.jpg -- broken data stream when reading image file
[BAD] diseased/Maize_leaf beetle/leaf beetle797_.jpg -- broken data stream when reading image file
[BAD] diseased/Maize_leaf blight/leaf blight379_.jpg -- broken data stream when reading image file
[BAD] diseased/Maize_leaf bligh

In [50]:
data

,filename,label
0,All_healthy/Cashew_healthy/healthy1000_.jpg,All_healthy_Cashew_healthy
1,All_healthy/Cashew_healthy/healthy1001_.jpg,All_healthy_Cashew_healthy
2,All_healthy/Cashew_healthy/healthy1002_.jpg,All_healthy_Cashew_healthy
3,All_healthy/Cashew_healthy/healthy1003_.jpg,All_healthy_Cashew_healthy
4,All_healthy/Cashew_healthy/healthy1004_.jpg,All_healthy_Cashew_healthy
...,...,...
25121,diseased/Tomato_verticulium wilt/verticulium w...,diseased_Tomato_verticulium wilt
25122,diseased/Tomato_verticulium wilt/verticulium w...,diseased_Tomato_verticulium wilt
25123,diseased/Tomato_verticulium wilt/verticulium w...,diseased_Tomato_verticulium wilt
25124,diseased/Tomato_verticulium wilt/verticulium w...,diseased_Tomato_verticulium wilt


In [51]:
from sklearn.model_selection import train_test_split

# Make sure paths are consistent
data['filename'] = data['filename'].apply(lambda x: x.replace("\\", "/"))

# Split into train (80%) and test (20%)
train_df, test_df = train_test_split(data, test_size=0.2, stratify=data['label'], random_state=42)


In [52]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Create data generators
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filename',
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_gen = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filename',
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

test_gen = datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filename',
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)


Found 16080 validated image filenames belonging to 22 classes.
Found 4020 validated image filenames belonging to 22 classes.
Found 5026 validated image filenames belonging to 22 classes.


In [53]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Load base model
base_model = MobileNetV2(input_shape=IMG_SIZE + (3,), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base model

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)

num_classes = len(train_gen.class_indices)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)


In [54]:
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [55]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)



c:\Users\Public\one\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
503/503 ━━━━━━━━━━━━━━━━━━━━ 496s 973ms/step - accuracy: 0.1953 - loss: 2.8228 - val_accuracy: 0.5983 - val_loss: 1.4094
Epoch 2/10
503/503 ━━━━━━━━━━━━━━━━━━━━ 391s 777ms/step - accuracy: 0.5514 - loss: 1.4071 - val_accuracy: 0.6831 - val_loss: 1.0370
Epoch 3/10
503/503 ━━━━━━━━━━━━━━━━━━━━ 396s 787ms/step - accuracy: 0.6509 - loss: 1.0856 - val_accuracy: 0.7075 - val_loss: 0.8944
Epoch 4/10
503/503 ━━━━━━━━━━━━━━━━━━━━ 407s 810ms/step - accuracy: 0.6973 - loss: 0.9258 - val_accuracy: 0.7294 - val_loss: 0.8127
Epoch 5/10
503/503 ━━━━━━━━━━━━━━━━━━━━ 392s 779ms/step - accuracy: 0.7123 - loss: 0.8362 - val_accuracy: 0.7386 - val_loss: 0.7685
Epoch 6/10
503/503 ━━━━━━━━━━━━━━━━━━━━ 383s 761ms/step - accuracy: 0.7308 - loss: 0.7854 - val_accuracy: 0.7463 - val_loss: 0.7293
Epoch 7/10
503/503 ━━━━━━━━━━━━━━━━━━━━ 382s 760ms/step - accuracy: 0.7453 - loss: 0.7451 - val_accuracy: 0.7478 - val_loss: 0.7033
Epoch 8/10
503/503 ━━━━━━━━━━━━━━━━━━━━ 379s 754ms/step - accuracy: 0.7554 -

In [56]:
# Evaluate
val_loss, val_acc = model.evaluate(val_gen)
print(f'Validation accuracy: {val_acc:.2f}')

# Save model
model.save('plant_disease_model.h5')





126/126 ━━━━━━━━━━━━━━━━━━━━ 76s 601ms/step - accuracy: 0.7694 - loss: 0.6497


Validation accuracy: 0.77


In [61]:
from tensorflow.keras.models import load_model

model = load_model("plant_disease_model.h5")  # Path to your trained model


In [62]:
from tensorflow.keras.preprocessing import image
import numpy as np

IMG_SIZE = (224, 224)  # Same as used during training

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0  # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array


In [63]:
img_path = "predict.png"  # Replace with your test image path
img_array = preprocess_image(img_path)

pred = model.predict(img_array)
predicted_index = np.argmax(pred[0])
confidence = np.max(pred[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [64]:
labels_map = {v: k for k, v in train_gen.class_indices.items()}
predicted_label = labels_map[predicted_index]

print(f"Prediction: {predicted_label} (Confidence: {confidence:.2f})")


Prediction: diseased_Cashew_anthracnose (Confidence: 0.73)


In [ ]:
%%writefile app.py
import os
from flask import Flask, render_template, request
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'static/uploads'

model = load_model('plant_disease_model.h5')  # Load your trained model
IMG_SIZE = (224, 224)  # adjust based on your model input

def process_image(img_path):
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

@app.route('/', methods=['GET', 'POST'])  # ✅ Matches form method and route
def index():
    prediction = None
    image_url = None

    if request.method == 'POST':
        img_file = request.files['image']
        if img_file:
            path = os.path.join(app.config['UPLOAD_FOLDER'], img_file.filename)
            img_file.save(path)

            img_data = process_image(path)
            pred = model.predict(img_data)
            label = np.argmax(pred)  # adjust based on your label format

            prediction = f"Predicted class: {label}"
            image_url = path

    return render_template('index.html', prediction=prediction, image_url=image_url)

if __name__ == '__main__':
    app.run(debug=True)


Overwriting app.py
